# Raw to Excel/Word format

Converts the raw format of the physician notes and word docs to word/excel format for easier evaluation.

In [ ]:
import json
from datetime import datetime
from pathlib import Path

import openpyxl
import pandas as pd
from docx import Document
from openpyxl.styles import Font
from pydantic import BaseModel, validator

from llm_discharge_summaries.schemas.mimic import PhysicianNote
from llm_discharge_summaries.schemas.rcp_guidelines import (
    AllergiesAndAdverseReaction,
    RCPGuidelines,
)

In [ ]:
OUTPUT_DIR = Path.cwd() / "outputs"
JSON_INPUT_DIR = OUTPUT_DIR / "llm_responses"
EXCEL_OUTPUT_DIR = OUTPUT_DIR / "human_readable_responses"
PHYSICIAN_NOTE_FPATH = (
    Path.cwd()
    / "inputs"
    / "physionet.org"
    / "files"
    / "mimiciii"
    / "1.4"
    / "physician_notes_mimic.csv"
)

In [ ]:
EXCEL_OUTPUT_DIR.mkdir(exist_ok=True)

In [ ]:
notes_df = pd.read_csv(PHYSICIAN_NOTE_FPATH)

## Helper funcs

In [ ]:
class DischargeSummaryRow(BaseModel):
    section: str
    field: str
    value: str = ""

    @validator("value", pre=True, always=True)
    def set_empty_default(cls, value, values):
        # If section and field exist but value empty set to fixed message
        if values.get("section") and values.get("field"):
            return value if value.strip() else "Information not found in notes"
        else:
            return value


def create_rows_from_values(
    section: str, field: str, values: list[str]
) -> list[DischargeSummaryRow]:
    return (
        [
            DischargeSummaryRow(section=section, field=field, value=value)
            for value in values
        ]
        if values
        else [DischargeSummaryRow(section=section, field=field)]
    )


def create_allergies_and_adverse_reaction_rows(
    allergies_and_adverse_reaction: list[AllergiesAndAdverseReaction],
) -> list[DischargeSummaryRow]:
    # Handling of allergies is different as it is a list of objects instead of list of strings
    if not allergies_and_adverse_reaction:
        return [
            DischargeSummaryRow(
                section="Allergies",
                field="Description of Reaction 0",
                value="No known drug allergies or adverse reactions",
            )
        ]

    rows = []
    for idx, allergy in enumerate(allergies_and_adverse_reaction):
        rows.extend(
            [
                DischargeSummaryRow(
                    section="Allergies",
                    field=f"Description of Reaction {idx}",
                    value=allergy.description_of_reaction,
                ),
                DischargeSummaryRow(
                    section="Allergies",
                    field=f"Causative Agent {idx}",
                    value=allergy.causative_agent,
                ),
            ]
        )
    return rows


def discharge_summary_to_df(discharge_summary: RCPGuidelines):
    sections = []

    sections.append(
        [
            DischargeSummaryRow(
                section="Patient Demographics",
                field="Patient Name",
                values=discharge_summary.patient_demographics.patient_name,
            ),
            DischargeSummaryRow(
                section="Patient Demographics",
                field="Date of Birth",
                values=discharge_summary.patient_demographics.date_of_birth,
            ),
            DischargeSummaryRow(
                section="Patient Demographics",
                field="Patient Address",
                values=discharge_summary.patient_demographics.patient_address,
            ),
            DischargeSummaryRow(
                section="Patient Demographics",
                field="NHS Number",
                values=discharge_summary.patient_demographics.nhs_number,
            ),
            DischargeSummaryRow(
                section="Patient Demographics",
                field="Safety Alert",
                values=discharge_summary.patient_demographics.safety_alerts,
            ),
        ]
    )

    sections.append(
        [
            DischargeSummaryRow(
                section="GP Practice",
                field="GP Name",
                value=discharge_summary.GP_practice.GP_name,
            ),
            DischargeSummaryRow(
                section="GP Practice",
                field="Gp Practice Details",
                value=discharge_summary.GP_practice.GP_practice_details,
            ),
        ]
    )

    sections.append(
        create_rows_from_values(
            section="Social Context",
            field="Social Context",
            values=discharge_summary.social_context.social_context,
        )
    )

    sections.append(
        [
            DischargeSummaryRow(
                section="Admission Details",
                field="Reason for Admission",
                value=discharge_summary.admission_details.reason_for_admission,
            ),
            DischargeSummaryRow(
                section="Admission Details",
                field="Date Time of Admission",
                value=discharge_summary.admission_details.date_time_of_admission,
            ),
            DischargeSummaryRow(
                section="Admission Details",
                field="Admission Method",
                value=discharge_summary.admission_details.admission_method,
            ),
        ]
        + create_rows_from_values(
            section="Admission Details",
            field="Relevant Past Medical and Mental Health History",
            values=discharge_summary.admission_details.relevant_past_medical_and_mental_health_history,
        )
    )

    sections.append(
        [
            DischargeSummaryRow(
                section="Diagnoses",
                field="Primary Diagnosis",
                value=discharge_summary.diagnoses.primary_diagnosis,
            ),
        ]
        + create_rows_from_values(
            section="Diagnoses",
            field="Secondary Diagnoses",
            values=discharge_summary.diagnoses.secondary_diagnoses,
        )
    )

    sections.append(
        [
            DischargeSummaryRow(
                section="Clinical Summary",
                field="Clinical Summary",
                value=discharge_summary.clinical_summary.clinical_summary,
            )
        ]
        + create_rows_from_values(
            section="Clinical Summary",
            field="Procedures",
            values=discharge_summary.clinical_summary.procedures,
        )
        + create_rows_from_values(
            section="Clinical Summary",
            field="Investigation Results",
            values=discharge_summary.clinical_summary.investigation_results,
        )
    )

    sections.append(
        [
            DischargeSummaryRow(
                section="Discharge Details",
                field="Date Time of Discharge",
                value=discharge_summary.discharge_details.date_time_of_discharge,
            ),
            DischargeSummaryRow(
                section="Discharge Details",
                field="Discharge Destination",
                value=discharge_summary.discharge_details.discharge_destination,
            ),
        ]
    )

    sections.append(
        create_rows_from_values(
            section="Plan and Requested Actions",
            field="Post Discharge Plan And Requested Actions",
            values=discharge_summary.plan_and_requested_actions.post_discharge_plan_and_requested_actions,
        )
        + create_rows_from_values(
            section="Plan and Requested Actions",
            field="Information And Advice Given",
            values=discharge_summary.plan_and_requested_actions.information_and_advice_given,
        )
        + create_rows_from_values(
            section="Plan and Requested Actions",
            field="Patient And Carer Concerns Expectations And Wishes",
            values=discharge_summary.plan_and_requested_actions.patient_and_carer_concerns_expectations_and_wishes,
        )
        + [
            DischargeSummaryRow(
                section="Plan and Requested Actions",
                field="Next Appointment Details",
                value=discharge_summary.plan_and_requested_actions.next_appointment_details,
            )
        ]
    )

    sections.append(
        create_allergies_and_adverse_reaction_rows(
            discharge_summary.allergies_and_adverse_reaction
        )
    )

    # Flatten and add a spacer between sections
    rows = []
    for section in sections:
        rows.extend([row.dict() for row in section])
        rows.append(DischargeSummaryRow(section="", field="", value="").dict())

    # Create df and do some basic formatting
    df = pd.DataFrame.from_dict(rows)
    df["section"] = df["section"].drop_duplicates(keep="first")
    df["field"] = df["field"].drop_duplicates(keep="first")
    df.fillna("", inplace=True)
    df.columns = df.columns.str.capitalize()
    return df

In [ ]:
def format_excel(ws):
    ws.insert_rows(1)
    ws.cell(row=1, column=1, value="GPT Discharge Summary")
    ws.cell(row=1, column=5, value="Evaluation")

    for idx, eval_heading in enumerate(
        [
            "Missed- Severe",
            "Missed- Minor",
            "Added- Hallucination",
            "Added- Not relevant",
            "Explanation of Error",
            "Comments",
        ]
    ):
        ws.cell(row=2, column=5 + idx, value=eval_heading)

    for c in ws["A"]:
        c.font = Font(bold=True, sz=11)
    for cell in ws.iter_rows(min_row=2, max_row=2, values_only=True):
        c.font = Font(bold=True, sz=12)
    for cell in ws.iter_rows(min_row=1, max_row=1, values_only=True):
        c.font = Font(bold=True, sz=14)

    for column in ws.columns:
        ws.column_dimensions[column[0].column_letter].width = 40
    ws.column_dimensions["C"].width = 80

    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = openpyxl.styles.Alignment(wrap_text=True)

In [ ]:
def discharge_summary_to_excel(discharge_summary: RCPGuidelines, excel_fpath: Path):
    rows_df = discharge_summary_to_df(discharge_summary)
    rows_df.to_excel(excel_fpath, index=False)

    wb = openpyxl.load_workbook(excel_fpath)
    format_excel(wb.active)
    wb.save(excel_fpath)

In [ ]:
def physician_notes_to_word(physician_notes: list[PhysicianNote], word_fpath):
    doc = Document()

    doc.add_heading(f"Physician Notes Patient ID {physician_notes[0].hadm_id}", level=1)
    for note in physician_notes:
        date_uk_format = datetime.strptime(
            note.timestamp, "%Y-%m-%d %H:%M:%S"
        ).strftime("%d-%m-%Y %H:%M:%S")
        doc.add_heading(f"{note.title}: {date_uk_format}", level=2)
        doc.add_paragraph(note.text)

    doc.save(word_fpath)

## Write to file


Manual fix for example that does not follow json schema

In [ ]:
# faulty_json_path = LLM_OUTPUT_DIR / "179544" / "discharge_summary.json"
# faulty_json = json.loads(faulty_json_path.read_text())
# faulty_json["GP_practice"] = {"GP_name": ""}
# faulty_json_path.write_text(json.dumps(faulty_json, indent=4))


For each GPT produced JSON convert it to excel to allow for easier annotation. Do the same for the associated physician notes but to word

In [ ]:
for hadm_id_output_dir in JSON_INPUT_DIR.iterdir():
    if not hadm_id_output_dir.is_dir():
        continue

    hadm_id = hadm_id_output_dir.stem

    human_output_hadm_id_dir = EXCEL_OUTPUT_DIR / hadm_id
    human_output_hadm_id_dir.mkdir(exist_ok=True)

    discharge_summary = RCPGuidelines(
        **json.loads((hadm_id_output_dir / "discharge_summary.json").read_text())
    )
    discharge_summary_to_excel(
        discharge_summary,
        human_output_hadm_id_dir / f"discharge_summary_{hadm_id}.xlsx",
    )

    physician_notes = [
        PhysicianNote(
            hadm_id=row["HADM_ID"],
            title=row["DESCRIPTION"],
            timestamp=row["CHARTTIME"],
            text=row["TEXT"],
        )
        for _, row in notes_df[notes_df["HADM_ID"] == int(hadm_id)].iterrows()
    ]
    physician_notes_to_word(
        physician_notes, human_output_hadm_id_dir / f"physician_notes_{hadm_id}.docx"
    )